# Autolabel TMA Cores

## This notebook is an example: create a copy before running it or you will get merge conflicts!

**NOTE**: Before running this notebook for the first time, make sure you've coregistered your instrument using the *update coregistration parameters* section of  `1_set_up_toffy.ipynb`. This will ensure your FOVs display correctly on the slide.



### Background
This notebook automatically checks the names assigned to the cores on a TMA. In order to get the most benefit out of the notebook, make sure that you've named your FOVs appropriately. The expected format is RNCM, where N is the row and M is the column of the TMA. For example, a core on the third row and second column would be R3C2, and one on the 7th row and first column would be R7C1.  

The script expects that you have already generated and moved the necessary files into the appropriate directory before starting.
- A JSON file defining the four corners of the TMA. It's important that you have selected them in the correct order; top left, top right, bottom left, bottom right. Even if one of the cores on the corner is missing, make sure the FOV is located where that corner of the TMA *would* be located, as this is used to define the dimensions of the TMA. You can create this file by exporting the FOVs from the MIBIControl software. 
- A JSON file containing all of the FOVs that you have selected from the TMA, named appropriately. You can create this file by exporting the FOVs from the MIBIControl software. 
- The optical image of your TMA slide. This is automatically created when you load your slide, and is saved to the *Data/optical-image* subfolder

In [2]:
import sys
sys.path.append('../')

In [3]:
import json
import os
from skimage.io import imread

from toffy import tiling_utils, json_utils

# suppress mpl deprecation
import warnings
from matplotlib.cbook import mplDeprecation
warnings.filterwarnings("ignore", category=mplDeprecation)

## 1. Copy over the necessary files to start the script

Set *tma_dir*, the name of the directory to place and write all of your files. This directory should be set to *C:\\\\Users\\\\Customer.ION\\\\Documents\\\\autolabeled_tma_jsons*. <b>Do not change this path</b>.

### After running this section, the directory structure should look like:

```
C:\\Users\\Customer.ION\\Documents\\autolabeled_tma_jsons
│ tma_corners_path (JSON file containing the FOVs defining the four corners of the TMA)
│ manual_run_path (JSON file defining the desired FOV locations and names on your TMA)
│ slide_path (image file containing the slide to generate FOVs on)
```

In [ ]:
tma_dir = 'C:\\Users\\Customer.ION\\Documents\\autolabeled_tma_jsons'

Define a prefix to use for the input and output files to place in *tma_dir*. <b>All files names must use this prefix</b>.

The default is *'example_tma'*, but you should change it something relevant to your study, such as *'BRCA_TMA_1'*

In [ ]:
tma_prefix = 'example_tma'

Set the path to the JSON defining the 4 corner points of the TMA.

This file is generated using MIBIcontrol. <b>It is important that they are selected in this order: top-left, top-right, bottom-left, bottom-right</b>. Name these FOVs per MIBI standards (*RNCM*, N is the row and M is the column on the TMA). <b>If one of the corners is missing, define the FOV at the location where the corner <i>would</i> be located</b>.

Move the downloaded file to *tma_dir*, and rename it *{tma_prefix}_corners.json*.

In [ ]:
tma_corners_path = os.path.join(tma_dir, '%s_corners.json' % tma_prefix)

Set the path to the JSON defining the desired locations and names of each FOV to generate on the TMA.

This file is generated using MIBIcontrol. <b>The same MIBI-naming standards as outlined for *tma_corners_path* should be applied here.</b>

Move the downloaded file to *tma_dir*, and rename it *{tma_prefix}_manual.json*.

In [ ]:
manual_run_path = os.path.join(tma_dir, '%s_manual.json' % tma_prefix)

Set the path to the slide image defining the TMA. This image can be found at *D:\\\Data\\\optical-image*.

Move the file to *json_tiling_dir* and rename it *{tiled_region_prefix}_slide.{slide_path_ext}*.

In [ ]:
# Change `slide_path_ext` to '.jpg' or '.png' if needed.
slide_path_ext = '.bmp' 

In [ ]:
slide_path = os.path.join(tma_dir, tma_prefix + slide_path_ext)

Set the path in *tma_dir* to write the final run JSON containing the FOVs to generate on the TMA.

In [ ]:
remapped_fov_path = os.path.join(tma_dir, '%s_automatic_run.json' % tma_prefix)

These files define automated files that will be created by the notebook. <b>Do not change these</b>.

In [ ]:
auto_fov_names_path = os.path.join(tma_dir, '%s_automatic_fov_names.json' % tma_prefix)
mapping_path = os.path.join(tma_dir, '%s_mapping.json' % tma_prefix)

## 2. Generate the automatic mapping of FOV names

In [ ]:
tma_num_row = 7
tma_num_col = 4

In [ ]:
# generate automatically named TMA
auto_fov_regions = tiling_utils.generate_tma_fov_list(
    tma_corners_path,
    tma_num_row,
    tma_num_col
)

# save the automatically-named TMA FOVs to centroids mapping
json_utils.write_json_file(json_path=auto_fov_names_path, json_object=auto_fov_regions, encoding='utf-8')

In [ ]:
# load the user-defined set of FOVs in
json_utils.read_json_file(manual_run_path, encoding='utf-8')

# ensure missing and duplicate FOV names get identified
manual_fov_regions = json_utils.rename_missing_fovs(manual_fov_regions)
manual_fov_regions = json_utils.rename_duplicate_fovs(manual_fov_regions)

# ensure all the provided FOVs in manual_fov_regions lie on the slide
for mf in manual_fov_regions['fovs']:
    mf_coord = (mf['centerPointMicrons']['x'], mf['centerPointMicrons']['y'])
    
    if not tiling_utils.verify_coordinate_on_slide(mf_coord, 'micron'):
        raise ValueError('FOV %s in manual_fov_regions is out of bounds' % mf['name'])

In [ ]:
# load the slide image in
slide_data = imread(slide_path)

## 3. Set thresholds for identifying incorrect FOV names

### The variables below control the tolerance for identifying when a core has been named incorrectly.

* `check_dist`: set to a positive value to notify of FOV mappings at a distance greater than this value (measured in microns), sorted by decreasing distance. Set to `None` to bypass.
* `check_duplicates`: set to `True` to flag FOVs in `auto_fov_regions` with multiple FOVs mapping to it. Set to `False` to bypass.
* `check_mismatches`: set to `True` to flag FOVs with mismatched names. Set to `False` to bypass. Assumes FOVs have been named R1C1, R1C2, etc. 

In [ ]:
check_dist = 2000
check_duplicates = True
check_mismatches = True

Each FOV in *manual_fov_regions* are mapped to their closest corresponding FOV in *auto_fov_regions* by default. To see the current mappings, select FOVs in the *Manually-defined FOV* menu. To remap a manual FOV to a different auto FOV, use the *Automatically-generated FOV* menu.

<b>NOTE: if no remapping needs to be done, click <i>Save mapping</i> to generate the default mapping for the next step.</b>

After you're done finished, click *Save mapping* and run the cells afterward. <b>Ignore any error messages that may appear there before this happens</b>. You can always come back here and redo your mappings if you change your mind.

In [ ]:
%matplotlib widget
tiling_utils.tma_interactive_remap(
    manual_fov_regions,
    auto_fov_regions,
    slide_data,
    mapping_path,
    draw_radius=7,
    figsize=(7, 7),
    check_dist=check_dist,
    check_duplicates=check_duplicates,
    check_mismatches=check_mismatches
)

## 4. Set parameters for remapped JSON

Set `randomize` to `True` to shuffle the order of the FOVs in `remapped_fov_regions`. This avoids potential batch effects of acquisition order.

In [ ]:
randomize = True

In [ ]:
# load the mapping saved by the interactive visualization
mapping = json_utils.read_json_file(mapping_path, encoding='utf-8')

# rename FOVs, randomize the order, and insert moly points at a specified interval
remapped_fov_regions = tiling_utils.remap_and_reorder_fovs(
    manual_fov_regions,
    mapping,
    randomize=randomize
)

# save remapped_fov_regions
json_utils.write_json_file(json_path=remapped_fov_path, json_object=remapped_fov_regions, encoding='utf-8')

## 5. Run JSON adjustment

If you would like to break up your tma file into smaller JSONs containing a specified amount of FOVs in each, you can do that using the code below. 

`file_split` is a list of values detailing how many FOVs to included in each new file, and must sum to the total number of FOVs in the run file

In [5]:
file_split = []

json_utils.split_run_file(tma_dir, os.path.basename(remapped_fov_path), file_split)